# Query Operations

Objectives:
 * Learn how to use the query method for tabular datasets
 
Note: this notebook only works with HDF Server!

In [ ]:
%matplotlib inline
import h5pyd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# Open a file containing stock quote data
f = h5pyd.File("hdf5://shared/sample/snp500.h5", 'r')

In [ ]:
dset = f["dset"]
type(dset)  # The Table class is a one-dimensional dataset with compound type

In [ ]:
dset.nrows

In [ ]:
# dtype is a list of field names and sub-types
dset.dtype

In [ ]:
# The date field starting in 1970
arr = dset[:10]  # get first 10 elements
arr['date']  # date starts in 1970

In [ ]:
arr = dset[-10:]  # get last 10 elements
arr['date']  # and ends in 2015

In [ ]:
# If we wanted to extract all stock quotes with the symbol AAPL
# We could read the dataset in chunks and filter out anything 
# other than that symbol, but it would be rather slow
#
# More efficient is to use the dset query operator which can just 
# return rows matching the specification
symbol = "AAPL"
%time arr = dset.read_where(f"symbol == b'{symbol}'")

In [ ]:
arr.shape

In [ ]:
# convert numpy result to Pandas dataframe
df = pd.DataFrame(arr)
df

In [ ]:
# Calculate the 20 and 100 days moving averages of the closing prices
close = df['close']
short_rolling = close.rolling(window=20).mean()
long_rolling = close.rolling(window=100).mean()

# Plot everything by leveraging the very powerful matplotlib package
fig, ax = plt.subplots(figsize=(16,9))

ax.plot(close.index, close, label=symbol)
ax.plot(short_rolling.index, short_rolling, label='20 days rolling')
ax.plot(long_rolling.index, long_rolling, label='100 days rolling')

ax.set_xlabel('Date')
ax.set_ylabel('Adjusted closing price ($)')
ax.legend()

In [ ]:
df.describe()

In [ ]:
# find the largest one day gain for any stock
max_gain = 0.0
target = None
cursor = dset.create_cursor()
for row in cursor:
    gain = row["high"] - row["low"]
    if gain > max_gain:
        max_gain = gain
        target = np.copy(row)
print("largest gain:")
print(target)
    